# HTRU2 Dataset 
## with SMOTE

In [3]:
from sklearn import svm
from sklearn import metrics
import numpy as np
from sklearn.model_selection import KFold
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import BernoulliRBM
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.cluster import DBSCAN
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import silhouette_score
import sys
sys.path.append('/dartfs-hpc/rc/home/t/f00355t/imbalance')
from imblearn.over_sampling import SMOTE
np.set_printoptions(threshold='nan')
tmp = np.loadtxt("HTRU_2.csv", dtype=np.str,skiprows=1, delimiter=",")
total_data=tmp[0:,0:tmp.shape[1]-2].astype(np.float)
total_label=tmp[0:,tmp.shape[1]-1].astype(np.float)

import random
import math
def randomly_pos_points_pick(training_data, training_label, pos_points_ratio_power):
    positive_points_index_set = np.where(training_label==1)[0]
    num_to_pick = int(math.pow(10, pos_points_ratio_power) * len(training_label))
    # Need at least two pos points to apply SMOTE
    if num_to_pick < 2:
        num_to_pick = 2
    random.seed(0)
    # Use random.sample to pick up positive points randomly
    random_index_set = random.sample(list(positive_points_index_set), num_to_pick)
    # Use the index to filter out original training data and training label
    print random_index_set
    zero_data=training_data[np.where(training_label==0)[0]]
    zero_label=training_label[np.where(training_label==0)[0]]
    training_data_after_sampling = training_data[random_index_set]
    training_label_after_sampling = training_label[random_index_set]
    training_data_after_sampling=np.concatenate((zero_data, training_data_after_sampling), axis=0)
    training_label_after_sampling = np.concatenate((zero_label, training_label_after_sampling), axis=0)
    return training_data_after_sampling, training_label_after_sampling


skf = StratifiedKFold(n_splits=10)
kfold = skf.split(total_data, total_label)
K = -1
svm_matrix = np.zeros((10, 2, 2))
svm_auc = np.zeros((10, 1, 1))
svm_recall = np.zeros((10, 1, 1))
svm_p = np.zeros((10, 1, 1))
svm_F1 = np.zeros((10, 1, 1))

bn_matrix = np.zeros((10, 2, 2))
bn_auc = np.zeros((10, 1, 1))
bn_recall = np.zeros((10, 1, 1))
bn_p = np.zeros((10, 1, 1))
bn_F1 = np.zeros((10, 1, 1))


randomforest_matrix = np.zeros((10, 2, 2))
randomforest_auc = np.zeros((10, 1, 1))
randomforest_recall = np.zeros((10, 1, 1))
randomforest_p = np.zeros((10, 1, 1))
randomforest_F1 = np.zeros((10, 1, 1))

knn_matrix = np.zeros((10, 2, 2))
knn_auc = np.zeros((10, 1, 1))
knn_recall = np.zeros((10, 1, 1))
knn_p = np.zeros((10, 1, 1))
knn_F1 = np.zeros((10, 1, 1))

lr_matrix = np.zeros((10, 2, 2))
lr_auc = np.zeros((10, 1, 1))
lr_recall = np.zeros((10, 1, 1))
lr_p = np.zeros((10, 1, 1))
lr_F1 = np.zeros((10, 1, 1))

gb_matrix = np.zeros((10, 2, 2))
gb_auc = np.zeros((10, 1, 1))
gb_recall = np.zeros((10, 1, 1))
gb_p = np.zeros((10, 1, 1))
gb_F1 = np.zeros((10, 1, 1))
c_matrix=np.zeros((2,2))
c_matrix[1][1]=0.001
c_matrix[0][0]=0.001
c_matrix[1][0]=0.001
c_matrix[0][1]=0.001
for traini, testi in kfold:
	K = K + 1
	data_train, data_test = total_data[traini], total_data[testi]
	label_train, label_test = total_label[traini], total_label[testi]
	sm = SMOTE(random_state=1)
	data_train, label_train = sm.fit_sample(data_train, label_train)
	line = data_train.size / data_train[0].size

    #data_train, label2 = shuffle(data_train, label2, random_state=830)






	bn = BernoulliNB()
	predict_label =bn.fit(data_train, label_train).predict(data_test)
	df=bn.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	bn_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	bn_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	bn_F1[K]=2*(bn_recall[K]*bn_p[K])/(bn_recall[K]+bn_p[K])
	bn_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(bn_F1[K])):
		bn_F1[K] = 0


	knn2 = KNeighborsClassifier()
	predict_label =knn2.fit(data_train, label_train).predict(data_test)
	df=knn2.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	knn_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	knn_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	knn_F1[K]=2*(knn_recall[K]*knn_p[K])/(knn_recall[K]+knn_p[K])
	knn_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(knn_F1[K])):
		knn_F1[K] = 0


	lr2 = LogisticRegression()
	predict_label =lr2.fit(data_train, label_train).predict(data_test)
	df=lr2.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	lr_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	lr_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	lr_F1[K]=2*(lr_recall[K]*lr_p[K])/(lr_recall[K]+lr_p[K])
	lr_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(lr_F1[K])):
		lr_F1[K] = 0

	gb = GaussianNB()
	predict_label =gb.fit(data_train, label_train).predict(data_test)
	df=gb.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	gb_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	gb_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	gb_F1[K]=2*(gb_recall[K]*gb_p[K])/(gb_recall[K]+gb_p[K])
	gb_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(gb_F1[K])):
		gb_F1[K] = 0
	

	randomforest = RandomForestClassifier()
	predict_label = randomforest.fit(data_train, label_train).predict(data_test)
	df = randomforest.predict_proba(data_test)[:, 1]
	c_matrix= confusion_matrix(label_test, predict_label)
	randomforest_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	randomforest_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	randomforest_F1[K]=2*(randomforest_recall[K]*randomforest_p[K])/(randomforest_recall[K]+randomforest_p[K])
	randomforest_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(randomforest_F1[K])):
		randomforest_F1[K] = 0

print "----------------------Finally-------------------------"


print "bn"
print "The AUC is %0.3f " % (np.mean(bn_auc))
print "The recall is %0.3f " % (np.mean(bn_recall))
print "The p is %0.3f " % (np.mean(bn_p))
print "The F1 is %0.3f " % (np.mean(bn_F1))

print "knn"
print "The AUC is %0.3f " % (np.mean(knn_auc))
print "The recall is %0.3f " % (np.mean(knn_recall))
print "The p is %0.3f " % (np.mean(knn_p))
print "The F1 is %0.3f " % (np.mean(knn_F1))

print "lr"
print "The AUC is %0.3f " % (np.mean(lr_auc))
print "The recall is %0.3f " % (np.mean(lr_recall))
print "The p is %0.3f " % (np.mean(lr_p))
print "The F1 is %0.3f " % (np.mean(lr_F1))

print "gb"
print "The AUC is %0.3f " % (np.mean(gb_auc))
print "The recall is %0.3f " % (np.mean(gb_recall))
print "The p is %0.3f " % (np.mean(gb_p))
print "The F1 is %0.3f " % (np.mean(gb_F1))

print "randomforest"
print "The AUC is %0.3f " % (np.mean(randomforest_auc))
print "The recall is %0.3f " % (np.mean(randomforest_recall))
print "The p is %0.3f " % (np.mean(randomforest_p))
print "The F1 is %0.3f " % (np.mean(randomforest_F1))

----------------------Finally-------------------------
bn
The AUC is 0.764 
The recall is 0.973 
The p is 0.166 
The F1 is 0.284 
knn
The AUC is 0.937 
The recall is 0.876 
The p is 0.589 
The F1 is 0.703 
lr
The AUC is 0.976 
The recall is 0.908 
The p is 0.785 
The F1 is 0.840 
gb
The AUC is 0.958 
The recall is 0.855 
The p is 0.646 
The F1 is 0.730 
randomforest
The AUC is 0.964 
The recall is 0.882 
The p is 0.835 
The F1 is 0.856 


# AID687 Dataset with SMOTE

In [2]:
from sklearn import svm
from sklearn import metrics
import numpy as np  
from sklearn.model_selection import KFold
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import BernoulliRBM
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.cluster import DBSCAN
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import silhouette_score
import sys
sys.path.append('/dartfs-hpc/rc/home/t/f00355t/imbalance')
from imblearn.over_sampling import SMOTE
np.set_printoptions(threshold='nan')
tmp = np.loadtxt("AID687.csv", dtype=np.str,skiprows=1, delimiter=",")
total_data=tmp[0:,0:tmp.shape[1]-2].astype(np.float)
total_label=tmp[0:,tmp.shape[1]-1].astype(np.float)

import random
import math
def randomly_pos_points_pick(training_data, training_label, pos_points_ratio_power):
    positive_points_index_set = np.where(training_label==1)[0]
    num_to_pick = int(math.pow(10, pos_points_ratio_power) * len(training_label))
    # Need at least two pos points to apply SMOTE
    if num_to_pick < 2:
        num_to_pick = 2
    random.seed(0)
    # Use random.sample to pick up positive points randomly
    random_index_set = random.sample(list(positive_points_index_set), num_to_pick)
    # Use the index to filter out original training data and training label
    print random_index_set
    zero_data=training_data[np.where(training_label==0)[0]]
    zero_label=training_label[np.where(training_label==0)[0]]
    training_data_after_sampling = training_data[random_index_set]
    training_label_after_sampling = training_label[random_index_set]
    training_data_after_sampling=np.concatenate((zero_data, training_data_after_sampling), axis=0)
    training_label_after_sampling = np.concatenate((zero_label, training_label_after_sampling), axis=0)
    return training_data_after_sampling, training_label_after_sampling


skf = StratifiedKFold(n_splits=10)
kfold = skf.split(total_data, total_label)
K = -1
svm_matrix = np.zeros((10, 2, 2))
svm_auc = np.zeros((10, 1, 1))
svm_recall = np.zeros((10, 1, 1))
svm_p = np.zeros((10, 1, 1))
svm_F1 = np.zeros((10, 1, 1))

bn_matrix = np.zeros((10, 2, 2))
bn_auc = np.zeros((10, 1, 1))
bn_recall = np.zeros((10, 1, 1))
bn_p = np.zeros((10, 1, 1))
bn_F1 = np.zeros((10, 1, 1))


randomforest_matrix = np.zeros((10, 2, 2))
randomforest_auc = np.zeros((10, 1, 1))
randomforest_recall = np.zeros((10, 1, 1))
randomforest_p = np.zeros((10, 1, 1))
randomforest_F1 = np.zeros((10, 1, 1))

knn_matrix = np.zeros((10, 2, 2))
knn_auc = np.zeros((10, 1, 1))
knn_recall = np.zeros((10, 1, 1))
knn_p = np.zeros((10, 1, 1))
knn_F1 = np.zeros((10, 1, 1))

lr_matrix = np.zeros((10, 2, 2))
lr_auc = np.zeros((10, 1, 1))
lr_recall = np.zeros((10, 1, 1))
lr_p = np.zeros((10, 1, 1))
lr_F1 = np.zeros((10, 1, 1))

gb_matrix = np.zeros((10, 2, 2))
gb_auc = np.zeros((10, 1, 1))
gb_recall = np.zeros((10, 1, 1))
gb_p = np.zeros((10, 1, 1))
gb_F1 = np.zeros((10, 1, 1))
c_matrix=np.zeros((2,2))
c_matrix[1][1]=0.001
c_matrix[0][0]=0.001
c_matrix[1][0]=0.001
c_matrix[0][1]=0.001
for traini, testi in kfold:
	K = K + 1
	data_train, data_test = total_data[traini], total_data[testi]
	label_train, label_test = total_label[traini], total_label[testi]
	sm = SMOTE(random_state=1)
	data_train, label_train = sm.fit_sample(data_train, label_train)
	line = data_train.size / data_train[0].size

    #data_train, label2 = shuffle(data_train, label2, random_state=830)


	bn = BernoulliNB()
	predict_label =bn.fit(data_train, label_train).predict(data_test)
	df=bn.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	bn_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	bn_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	bn_F1[K]=2*(bn_recall[K]*bn_p[K])/(bn_recall[K]+bn_p[K])
	bn_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(bn_F1[K])):
		bn_F1[K] = 0


	knn2 = KNeighborsClassifier()
	predict_label =knn2.fit(data_train, label_train).predict(data_test)
	df=knn2.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	knn_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	knn_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	knn_F1[K]=2*(knn_recall[K]*knn_p[K])/(knn_recall[K]+knn_p[K])
	knn_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(knn_F1[K])):
		knn_F1[K] = 0


	lr2 = LogisticRegression()
	predict_label =lr2.fit(data_train, label_train).predict(data_test)
	df=lr2.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	lr_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	lr_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	lr_F1[K]=2*(lr_recall[K]*lr_p[K])/(lr_recall[K]+lr_p[K])
	lr_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(lr_F1[K])):
		lr_F1[K] = 0

	gb = GaussianNB()
	predict_label =gb.fit(data_train, label_train).predict(data_test)
	df=gb.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	gb_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	gb_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	gb_F1[K]=2*(gb_recall[K]*gb_p[K])/(gb_recall[K]+gb_p[K])
	gb_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(gb_F1[K])):
		gb_F1[K] = 0
	

	randomforest = RandomForestClassifier(n_estimators=1)
	predict_label = randomforest.fit(data_train, label_train).predict(data_test)
	df = randomforest.predict_proba(data_test)[:, 1]
	c_matrix= confusion_matrix(label_test, predict_label)
	randomforest_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	randomforest_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	randomforest_F1[K]=2*(randomforest_recall[K]*randomforest_p[K])/(randomforest_recall[K]+randomforest_p[K])
	randomforest_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(randomforest_F1[K])):
		randomforest_F1[K] = 0

print "----------------------Finally-------------------------"


print "bn"
print "The AUC is %0.3f " % (np.mean(bn_auc))
print "The recall is %0.3f " % (np.mean(bn_recall))
print "The p is %0.3f " % (np.mean(bn_p))
print "The F1 is %0.3f " % (np.mean(bn_F1))

print "knn"
print "The AUC is %0.3f " % (np.mean(knn_auc))
print "The recall is %0.3f " % (np.mean(knn_recall))
print "The p is %0.3f " % (np.mean(knn_p))
print "The F1 is %0.3f " % (np.mean(knn_F1))

print "lr"
print "The AUC is %0.3f " % (np.mean(lr_auc))
print "The recall is %0.3f " % (np.mean(lr_recall))
print "The p is %0.3f " % (np.mean(lr_p))
print "The F1 is %0.3f " % (np.mean(lr_F1))

print "gb"
print "The AUC is %0.3f " % (np.mean(gb_auc))
print "The recall is %0.3f " % (np.mean(gb_recall))
print "The p is %0.3f " % (np.mean(gb_p))
print "The F1 is %0.3f " % (np.mean(gb_F1))

print "randomforest"
print "The AUC is %0.3f " % (np.mean(randomforest_auc))
print "The recall is %0.3f " % (np.mean(randomforest_recall))
print "The p is %0.3f " % (np.mean(randomforest_p))
print "The F1 is %0.3f " % (np.mean(randomforest_F1))

d:\python27\lib\site-packages\ipykernel_launcher.py:121: RuntimeWarning: invalid value encountered in divide
d:\python27\lib\site-packages\ipykernel_launcher.py:168: RuntimeWarning: invalid value encountered in divide
d:\python27\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in divide


----------------------Finally-------------------------
bn
The AUC is 0.612 
The recall is 0.204 
The p is 0.004 
The F1 is 0.008 
knn
The AUC is 0.568 
The recall is 0.261 
The p is 0.005 
The F1 is 0.010 
lr
The AUC is 0.630 
The recall is 0.449 
The p is 0.005 
The F1 is 0.011 
gb
The AUC is 0.537 
The recall is 0.880 
The p is 0.003 
The F1 is 0.006 
randomforest
The AUC is 0.536 
The recall is 0.081 
The p is 0.027 
The F1 is 0.041 


# AID688 Dataset with SMOTE

In [5]:
from sklearn import svm
from sklearn import metrics
import numpy as np  
from sklearn.model_selection import KFold
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import BernoulliRBM
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.cluster import DBSCAN
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import silhouette_score
import sys
sys.path.append('/dartfs-hpc/rc/home/t/f00355t/imbalance')
from imblearn.over_sampling import SMOTE
np.set_printoptions(threshold='nan')
tmp = np.loadtxt("AID688.csv", dtype=np.str,skiprows=1, delimiter=",")
total_data=tmp[0:,0:tmp.shape[1]-2].astype(np.float)
total_label=tmp[0:,tmp.shape[1]-1].astype(np.float)

import random
import math
def randomly_pos_points_pick(training_data, training_label, pos_points_ratio_power):
    positive_points_index_set = np.where(training_label==1)[0]
    num_to_pick = int(math.pow(10, pos_points_ratio_power) * len(training_label))
    # Need at least two pos points to apply SMOTE
    if num_to_pick < 2:
        num_to_pick = 2
    random.seed(0)
    # Use random.sample to pick up positive points randomly
    random_index_set = random.sample(list(positive_points_index_set), num_to_pick)
    # Use the index to filter out original training data and training label
    print random_index_set
    zero_data=training_data[np.where(training_label==0)[0]]
    zero_label=training_label[np.where(training_label==0)[0]]
    training_data_after_sampling = training_data[random_index_set]
    training_label_after_sampling = training_label[random_index_set]
    training_data_after_sampling=np.concatenate((zero_data, training_data_after_sampling), axis=0)
    training_label_after_sampling = np.concatenate((zero_label, training_label_after_sampling), axis=0)
    return training_data_after_sampling, training_label_after_sampling


skf = StratifiedKFold(n_splits=10)
kfold = skf.split(total_data, total_label)
K = -1
svm_matrix = np.zeros((10, 2, 2))
svm_auc = np.zeros((10, 1, 1))
svm_recall = np.zeros((10, 1, 1))
svm_p = np.zeros((10, 1, 1))
svm_F1 = np.zeros((10, 1, 1))

bn_matrix = np.zeros((10, 2, 2))
bn_auc = np.zeros((10, 1, 1))
bn_recall = np.zeros((10, 1, 1))
bn_p = np.zeros((10, 1, 1))
bn_F1 = np.zeros((10, 1, 1))


randomforest_matrix = np.zeros((10, 2, 2))
randomforest_auc = np.zeros((10, 1, 1))
randomforest_recall = np.zeros((10, 1, 1))
randomforest_p = np.zeros((10, 1, 1))
randomforest_F1 = np.zeros((10, 1, 1))

knn_matrix = np.zeros((10, 2, 2))
knn_auc = np.zeros((10, 1, 1))
knn_recall = np.zeros((10, 1, 1))
knn_p = np.zeros((10, 1, 1))
knn_F1 = np.zeros((10, 1, 1))

lr_matrix = np.zeros((10, 2, 2))
lr_auc = np.zeros((10, 1, 1))
lr_recall = np.zeros((10, 1, 1))
lr_p = np.zeros((10, 1, 1))
lr_F1 = np.zeros((10, 1, 1))

gb_matrix = np.zeros((10, 2, 2))
gb_auc = np.zeros((10, 1, 1))
gb_recall = np.zeros((10, 1, 1))
gb_p = np.zeros((10, 1, 1))
gb_F1 = np.zeros((10, 1, 1))
c_matrix=np.zeros((2,2))
c_matrix[1][1]=0.001
c_matrix[0][0]=0.001
c_matrix[1][0]=0.001
c_matrix[0][1]=0.001
for traini, testi in kfold:
	K = K + 1
	data_train, data_test = total_data[traini], total_data[testi]
	label_train, label_test = total_label[traini], total_label[testi]
	sm = SMOTE(random_state=1)
	data_train, label_train = sm.fit_sample(data_train, label_train)
	line = data_train.size / data_train[0].size

    #data_train, label2 = shuffle(data_train, label2, random_state=830)


	bn = BernoulliNB()
	predict_label =bn.fit(data_train, label_train).predict(data_test)
	df=bn.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	bn_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	bn_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	bn_F1[K]=2*(bn_recall[K]*bn_p[K])/(bn_recall[K]+bn_p[K])
	bn_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(bn_F1[K])):
		bn_F1[K] = 0


	knn2 = KNeighborsClassifier()
	predict_label =knn2.fit(data_train, label_train).predict(data_test)
	df=knn2.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	knn_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	knn_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	knn_F1[K]=2*(knn_recall[K]*knn_p[K])/(knn_recall[K]+knn_p[K])
	knn_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(knn_F1[K])):
		knn_F1[K] = 0


	lr2 = LogisticRegression()
	predict_label =lr2.fit(data_train, label_train).predict(data_test)
	df=lr2.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	lr_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	lr_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	lr_F1[K]=2*(lr_recall[K]*lr_p[K])/(lr_recall[K]+lr_p[K])
	lr_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(lr_F1[K])):
		lr_F1[K] = 0

	gb = GaussianNB()
	predict_label =gb.fit(data_train, label_train).predict(data_test)
	df=gb.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	gb_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	gb_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	gb_F1[K]=2*(gb_recall[K]*gb_p[K])/(gb_recall[K]+gb_p[K])
	gb_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(gb_F1[K])):
		gb_F1[K] = 0
	

	randomforest = RandomForestClassifier()
	predict_label = randomforest.fit(data_train, label_train).predict(data_test)
	df = randomforest.predict_proba(data_test)[:, 1]
	c_matrix= confusion_matrix(label_test, predict_label)
	randomforest_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	randomforest_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	randomforest_F1[K]=2*(randomforest_recall[K]*randomforest_p[K])/(randomforest_recall[K]+randomforest_p[K])
	randomforest_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(randomforest_F1[K])):
		randomforest_F1[K] = 0

print "----------------------Finally-------------------------"


print "bn"
print "The AUC is %0.3f " % (np.mean(bn_auc))
print "The recall is %0.3f " % (np.mean(bn_recall))
print "The p is %0.3f " % (np.mean(bn_p))
print "The F1 is %0.3f " % (np.mean(bn_F1))

print "knn"
print "The AUC is %0.3f " % (np.mean(knn_auc))
print "The recall is %0.3f " % (np.mean(knn_recall))
print "The p is %0.3f " % (np.mean(knn_p))
print "The F1 is %0.3f " % (np.mean(knn_F1))

print "lr"
print "The AUC is %0.3f " % (np.mean(lr_auc))
print "The recall is %0.3f " % (np.mean(lr_recall))
print "The p is %0.3f " % (np.mean(lr_p))
print "The F1 is %0.3f " % (np.mean(lr_F1))

print "gb"
print "The AUC is %0.3f " % (np.mean(gb_auc))
print "The recall is %0.3f " % (np.mean(gb_recall))
print "The p is %0.3f " % (np.mean(gb_p))
print "The F1 is %0.3f " % (np.mean(gb_F1))

print "randomforest"
print "The AUC is %0.3f " % (np.mean(randomforest_auc))
print "The recall is %0.3f " % (np.mean(randomforest_recall))
print "The p is %0.3f " % (np.mean(randomforest_p))
print "The F1 is %0.3f " % (np.mean(randomforest_F1))

d:\python27\lib\site-packages\ipykernel_launcher.py:168: RuntimeWarning: invalid value encountered in divide
d:\python27\lib\site-packages\ipykernel_launcher.py:167: RuntimeWarning: invalid value encountered in divide


----------------------Finally-------------------------
bn
The AUC is 0.540 
The recall is 0.209 
The p is 0.010 
The F1 is 0.019 
knn
The AUC is 0.506 
The recall is 0.221 
The p is 0.010 
The F1 is 0.019 
lr
The AUC is 0.559 
The recall is 0.435 
The p is 0.011 
The F1 is 0.022 
gb
The AUC is 0.516 
The recall is 0.887 
The p is 0.009 
The F1 is 0.018 
randomforest
The AUC is 0.493 
The recall is 0.000 
The p is nan 
The F1 is 0.000 


# AID373 daset with SMOTE

In [7]:
from sklearn import svm
from sklearn import metrics
import numpy as np  
from sklearn.model_selection import KFold
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import BernoulliRBM
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.cluster import DBSCAN
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import silhouette_score
import sys
sys.path.append('/dartfs-hpc/rc/home/t/f00355t/imbalance')
from imblearn.over_sampling import SMOTE
np.set_printoptions(threshold='nan')
tmp = np.loadtxt("AID373.csv", dtype=np.str,skiprows=1, delimiter=",")
total_data=tmp[0:,0:tmp.shape[1]-2].astype(np.float)
total_label=tmp[0:,tmp.shape[1]-1].astype(np.float)

import random
import math
def randomly_pos_points_pick(training_data, training_label, pos_points_ratio_power):
    positive_points_index_set = np.where(training_label==1)[0]
    num_to_pick = int(math.pow(10, pos_points_ratio_power) * len(training_label))
    # Need at least two pos points to apply SMOTE
    if num_to_pick < 2:
        num_to_pick = 2
    random.seed(0)
    # Use random.sample to pick up positive points randomly
    random_index_set = random.sample(list(positive_points_index_set), num_to_pick)
    # Use the index to filter out original training data and training label
    print random_index_set
    zero_data=training_data[np.where(training_label==0)[0]]
    zero_label=training_label[np.where(training_label==0)[0]]
    training_data_after_sampling = training_data[random_index_set]
    training_label_after_sampling = training_label[random_index_set]
    training_data_after_sampling=np.concatenate((zero_data, training_data_after_sampling), axis=0)
    training_label_after_sampling = np.concatenate((zero_label, training_label_after_sampling), axis=0)
    return training_data_after_sampling, training_label_after_sampling


skf = StratifiedKFold(n_splits=10)
kfold = skf.split(total_data, total_label)
K = -1
svm_matrix = np.zeros((10, 2, 2))
svm_auc = np.zeros((10, 1, 1))
svm_recall = np.zeros((10, 1, 1))
svm_p = np.zeros((10, 1, 1))
svm_F1 = np.zeros((10, 1, 1))

bn_matrix = np.zeros((10, 2, 2))
bn_auc = np.zeros((10, 1, 1))
bn_recall = np.zeros((10, 1, 1))
bn_p = np.zeros((10, 1, 1))
bn_F1 = np.zeros((10, 1, 1))


randomforest_matrix = np.zeros((10, 2, 2))
randomforest_auc = np.zeros((10, 1, 1))
randomforest_recall = np.zeros((10, 1, 1))
randomforest_p = np.zeros((10, 1, 1))
randomforest_F1 = np.zeros((10, 1, 1))

knn_matrix = np.zeros((10, 2, 2))
knn_auc = np.zeros((10, 1, 1))
knn_recall = np.zeros((10, 1, 1))
knn_p = np.zeros((10, 1, 1))
knn_F1 = np.zeros((10, 1, 1))

lr_matrix = np.zeros((10, 2, 2))
lr_auc = np.zeros((10, 1, 1))
lr_recall = np.zeros((10, 1, 1))
lr_p = np.zeros((10, 1, 1))
lr_F1 = np.zeros((10, 1, 1))

gb_matrix = np.zeros((10, 2, 2))
gb_auc = np.zeros((10, 1, 1))
gb_recall = np.zeros((10, 1, 1))
gb_p = np.zeros((10, 1, 1))
gb_F1 = np.zeros((10, 1, 1))
c_matrix=np.zeros((2,2))
c_matrix[1][1]=0.001
c_matrix[0][0]=0.001
c_matrix[1][0]=0.001
c_matrix[0][1]=0.001
for traini, testi in kfold:
	K = K + 1
	data_train, data_test = total_data[traini], total_data[testi]
	label_train, label_test = total_label[traini], total_label[testi]
	sm = SMOTE(random_state=1)
	data_train, label_train = sm.fit_sample(data_train, label_train)
	line = data_train.size / data_train[0].size

    #data_train, label2 = shuffle(data_train, label2, random_state=830)


	bn = BernoulliNB()
	predict_label =bn.fit(data_train, label_train).predict(data_test)
	df=bn.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	bn_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	bn_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	bn_F1[K]=2*(bn_recall[K]*bn_p[K])/(bn_recall[K]+bn_p[K])
	bn_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(bn_F1[K])):
		bn_F1[K] = 0


	knn2 = KNeighborsClassifier()
	predict_label =knn2.fit(data_train, label_train).predict(data_test)
	df=knn2.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	knn_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	knn_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	knn_F1[K]=2*(knn_recall[K]*knn_p[K])/(knn_recall[K]+knn_p[K])
	knn_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(knn_F1[K])):
		knn_F1[K] = 0


	lr2 = LogisticRegression()
	predict_label =lr2.fit(data_train, label_train).predict(data_test)
	df=lr2.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	lr_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	lr_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	lr_F1[K]=2*(lr_recall[K]*lr_p[K])/(lr_recall[K]+lr_p[K])
	lr_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(lr_F1[K])):
		lr_F1[K] = 0

	gb = GaussianNB()
	predict_label =gb.fit(data_train, label_train).predict(data_test)
	df=gb.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	gb_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	gb_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	gb_F1[K]=2*(gb_recall[K]*gb_p[K])/(gb_recall[K]+gb_p[K])
	gb_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(gb_F1[K])):
		gb_F1[K] = 0
	

	randomforest = RandomForestClassifier()
	predict_label = randomforest.fit(data_train, label_train).predict(data_test)
	df = randomforest.predict_proba(data_test)[:, 1]
	c_matrix= confusion_matrix(label_test, predict_label)
	randomforest_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	randomforest_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	randomforest_F1[K]=2*(randomforest_recall[K]*randomforest_p[K])/(randomforest_recall[K]+randomforest_p[K])
	randomforest_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(randomforest_F1[K])):
		randomforest_F1[K] = 0

print "----------------------Finally-------------------------"


print "bn"
print "The AUC is %0.3f " % (np.mean(bn_auc))
print "The recall is %0.3f " % (np.mean(bn_recall))
print "The p is %0.3f " % (np.mean(bn_p))
print "The F1 is %0.3f " % (np.mean(bn_F1))

print "knn"
print "The AUC is %0.3f " % (np.mean(knn_auc))
print "The recall is %0.3f " % (np.mean(knn_recall))
print "The p is %0.3f " % (np.mean(knn_p))
print "The F1 is %0.3f " % (np.mean(knn_F1))

print "lr"
print "The AUC is %0.3f " % (np.mean(lr_auc))
print "The recall is %0.3f " % (np.mean(lr_recall))
print "The p is %0.3f " % (np.mean(lr_p))
print "The F1 is %0.3f " % (np.mean(lr_F1))

print "gb"
print "The AUC is %0.3f " % (np.mean(gb_auc))
print "The recall is %0.3f " % (np.mean(gb_recall))
print "The p is %0.3f " % (np.mean(gb_p))
print "The F1 is %0.3f " % (np.mean(gb_F1))

print "randomforest"
print "The AUC is %0.3f " % (np.mean(randomforest_auc))
print "The recall is %0.3f " % (np.mean(randomforest_recall))
print "The p is %0.3f " % (np.mean(randomforest_p))
print "The F1 is %0.3f " % (np.mean(randomforest_F1))

d:\python27\lib\site-packages\ipykernel_launcher.py:121: RuntimeWarning: invalid value encountered in divide
d:\python27\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in divide
d:\python27\lib\site-packages\ipykernel_launcher.py:145: RuntimeWarning: invalid value encountered in divide
d:\python27\lib\site-packages\ipykernel_launcher.py:156: RuntimeWarning: invalid value encountered in divide
d:\python27\lib\site-packages\ipykernel_launcher.py:167: RuntimeWarning: invalid value encountered in divide
d:\python27\lib\site-packages\ipykernel_launcher.py:168: RuntimeWarning: invalid value encountered in divide


----------------------Finally-------------------------
bn
The AUC is 0.609 
The recall is 0.250 
The p is 0.001 
The F1 is 0.002 
knn
The AUC is 0.590 
The recall is 0.200 
The p is 0.002 
The F1 is 0.004 
lr
The AUC is 0.713 
The recall is 0.200 
The p is 0.003 
The F1 is 0.005 
gb
The AUC is 0.459 
The recall is 0.400 
The p is 0.000 
The F1 is 0.000 
randomforest
The AUC is 0.648 
The recall is 0.100 
The p is nan 
The F1 is 0.100 


# AID604 daset with SMOTE

In [8]:
from sklearn import svm
from sklearn import metrics
import numpy as np  
from sklearn.model_selection import KFold
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import BernoulliRBM
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.cluster import DBSCAN
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import silhouette_score
import sys
sys.path.append('/dartfs-hpc/rc/home/t/f00355t/imbalance')
from imblearn.over_sampling import SMOTE
np.set_printoptions(threshold='nan')
tmp = np.loadtxt("AID604.csv", dtype=np.str,skiprows=1, delimiter=",")
total_data=tmp[0:,0:tmp.shape[1]-2].astype(np.float)
total_label=tmp[0:,tmp.shape[1]-1].astype(np.float)

import random
import math
def randomly_pos_points_pick(training_data, training_label, pos_points_ratio_power):
    positive_points_index_set = np.where(training_label==1)[0]
    num_to_pick = int(math.pow(10, pos_points_ratio_power) * len(training_label))
    # Need at least two pos points to apply SMOTE
    if num_to_pick < 2:
        num_to_pick = 2
    random.seed(0)
    # Use random.sample to pick up positive points randomly
    random_index_set = random.sample(list(positive_points_index_set), num_to_pick)
    # Use the index to filter out original training data and training label
    print random_index_set
    zero_data=training_data[np.where(training_label==0)[0]]
    zero_label=training_label[np.where(training_label==0)[0]]
    training_data_after_sampling = training_data[random_index_set]
    training_label_after_sampling = training_label[random_index_set]
    training_data_after_sampling=np.concatenate((zero_data, training_data_after_sampling), axis=0)
    training_label_after_sampling = np.concatenate((zero_label, training_label_after_sampling), axis=0)
    return training_data_after_sampling, training_label_after_sampling


skf = StratifiedKFold(n_splits=10)
kfold = skf.split(total_data, total_label)
K = -1
svm_matrix = np.zeros((10, 2, 2))
svm_auc = np.zeros((10, 1, 1))
svm_recall = np.zeros((10, 1, 1))
svm_p = np.zeros((10, 1, 1))
svm_F1 = np.zeros((10, 1, 1))

bn_matrix = np.zeros((10, 2, 2))
bn_auc = np.zeros((10, 1, 1))
bn_recall = np.zeros((10, 1, 1))
bn_p = np.zeros((10, 1, 1))
bn_F1 = np.zeros((10, 1, 1))


randomforest_matrix = np.zeros((10, 2, 2))
randomforest_auc = np.zeros((10, 1, 1))
randomforest_recall = np.zeros((10, 1, 1))
randomforest_p = np.zeros((10, 1, 1))
randomforest_F1 = np.zeros((10, 1, 1))

knn_matrix = np.zeros((10, 2, 2))
knn_auc = np.zeros((10, 1, 1))
knn_recall = np.zeros((10, 1, 1))
knn_p = np.zeros((10, 1, 1))
knn_F1 = np.zeros((10, 1, 1))

lr_matrix = np.zeros((10, 2, 2))
lr_auc = np.zeros((10, 1, 1))
lr_recall = np.zeros((10, 1, 1))
lr_p = np.zeros((10, 1, 1))
lr_F1 = np.zeros((10, 1, 1))

gb_matrix = np.zeros((10, 2, 2))
gb_auc = np.zeros((10, 1, 1))
gb_recall = np.zeros((10, 1, 1))
gb_p = np.zeros((10, 1, 1))
gb_F1 = np.zeros((10, 1, 1))
c_matrix=np.zeros((2,2))
c_matrix[1][1]=0.001
c_matrix[0][0]=0.001
c_matrix[1][0]=0.001
c_matrix[0][1]=0.001
for traini, testi in kfold:
	K = K + 1
	data_train, data_test = total_data[traini], total_data[testi]
	label_train, label_test = total_label[traini], total_label[testi]
	sm = SMOTE(random_state=1)
	data_train, label_train = sm.fit_sample(data_train, label_train)
	line = data_train.size / data_train[0].size

    #data_train, label2 = shuffle(data_train, label2, random_state=830)


	bn = BernoulliNB()
	predict_label =bn.fit(data_train, label_train).predict(data_test)
	df=bn.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	bn_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	bn_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	bn_F1[K]=2*(bn_recall[K]*bn_p[K])/(bn_recall[K]+bn_p[K])
	bn_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(bn_F1[K])):
		bn_F1[K] = 0


	knn2 = KNeighborsClassifier()
	predict_label =knn2.fit(data_train, label_train).predict(data_test)
	df=knn2.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	knn_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	knn_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	knn_F1[K]=2*(knn_recall[K]*knn_p[K])/(knn_recall[K]+knn_p[K])
	knn_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(knn_F1[K])):
		knn_F1[K] = 0


	lr2 = LogisticRegression()
	predict_label =lr2.fit(data_train, label_train).predict(data_test)
	df=lr2.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	lr_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	lr_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	lr_F1[K]=2*(lr_recall[K]*lr_p[K])/(lr_recall[K]+lr_p[K])
	lr_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(lr_F1[K])):
		lr_F1[K] = 0

	gb = GaussianNB()
	predict_label =gb.fit(data_train, label_train).predict(data_test)
	df=gb.predict_proba(data_test)[:,1]
	c_matrix= confusion_matrix(label_test, predict_label)
	gb_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	gb_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	gb_F1[K]=2*(gb_recall[K]*gb_p[K])/(gb_recall[K]+gb_p[K])
	gb_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(gb_F1[K])):
		gb_F1[K] = 0
	

	randomforest = RandomForestClassifier()
	predict_label = randomforest.fit(data_train, label_train).predict(data_test)
	df = randomforest.predict_proba(data_test)[:, 1]
	c_matrix= confusion_matrix(label_test, predict_label)
	randomforest_recall[K]=c_matrix[1,1]/(c_matrix[1,0]+c_matrix[1,1]+0.0)
	randomforest_p[K]=c_matrix[1,1]/(c_matrix[0,1]+c_matrix[1,1]+0.0)
	randomforest_F1[K]=2*(randomforest_recall[K]*randomforest_p[K])/(randomforest_recall[K]+randomforest_p[K])
	randomforest_auc[K]=roc_auc_score(label_test, df)
	if (np.isnan(randomforest_F1[K])):
		randomforest_F1[K] = 0

print "----------------------Finally-------------------------"


print "bn"
print "The AUC is %0.3f " % (np.mean(bn_auc))
print "The recall is %0.3f " % (np.mean(bn_recall))
print "The p is %0.3f " % (np.mean(bn_p))
print "The F1 is %0.3f " % (np.mean(bn_F1))

print "knn"
print "The AUC is %0.3f " % (np.mean(knn_auc))
print "The recall is %0.3f " % (np.mean(knn_recall))
print "The p is %0.3f " % (np.mean(knn_p))
print "The F1 is %0.3f " % (np.mean(knn_F1))

print "lr"
print "The AUC is %0.3f " % (np.mean(lr_auc))
print "The recall is %0.3f " % (np.mean(lr_recall))
print "The p is %0.3f " % (np.mean(lr_p))
print "The F1 is %0.3f " % (np.mean(lr_F1))

print "gb"
print "The AUC is %0.3f " % (np.mean(gb_auc))
print "The recall is %0.3f " % (np.mean(gb_recall))
print "The p is %0.3f " % (np.mean(gb_p))
print "The F1 is %0.3f " % (np.mean(gb_F1))

print "randomforest"
print "The AUC is %0.3f " % (np.mean(randomforest_auc))
print "The recall is %0.3f " % (np.mean(randomforest_recall))
print "The p is %0.3f " % (np.mean(randomforest_p))
print "The F1 is %0.3f " % (np.mean(randomforest_F1))

d:\python27\lib\site-packages\ipykernel_launcher.py:168: RuntimeWarning: invalid value encountered in divide


----------------------Finally-------------------------
bn
The AUC is 0.636 
The recall is 0.250 
The p is 0.005 
The F1 is 0.009 
knn
The AUC is 0.586 
The recall is 0.282 
The p is 0.008 
The F1 is 0.015 
lr
The AUC is 0.788 
The recall is 0.665 
The p is 0.011 
The F1 is 0.022 
gb
The AUC is 0.572 
The recall is 0.920 
The p is 0.004 
The F1 is 0.008 
randomforest
The AUC is 0.684 
The recall is 0.052 
The p is 0.295 
The F1 is 0.084 
